#### Cleaning Players Data

In [1]:
import pandas as pd
import regex as re
from datetime import datetime    
from unidecode import unidecode

In [2]:
players = pd.read_csv("2021.csv")

In [3]:
players["Year"] = 2021

In [4]:
players

,League,Player Link,Player,Position,Date of Birth,Country,Join,Contract,Market Value,Club,Club Link,Season,Year
0,Liga Portugal,https://www.transfermarkt.co.uk/babacar-niasse...,Babacar Niasse,Goalkeeper,"Dec 20, 1996 (24)",Mauritania,"Aug 21, 2019","Jun 30, 2023",€350k,CD Tondela,https://www.transfermarkt.co.uk/cd-tondela/sta...,https://www.transfermarkt.co.uk/liga-portugal/...,2021
1,Liga Portugal,https://www.transfermarkt.co.uk/pedro-trigueir...,Pedro Trigueira,Goalkeeper,"Jan 4, 1988 (33)",Portugal,"Aug 17, 2020","Jun 30, 2024",€300k,CD Tondela,https://www.transfermarkt.co.uk/cd-tondela/sta...,https://www.transfermarkt.co.uk/liga-portugal/...,2021
2,Liga Portugal,https://www.transfermarkt.co.uk/joel-sousa/pro...,Joel Sousa,Goalkeeper,"Mar 17, 2000 (21)",Portugal,"Jul 1, 2019","Jun 30, 2024",€100k,CD Tondela,https://www.transfermarkt.co.uk/cd-tondela/sta...,https://www.transfermarkt.co.uk/liga-portugal/...,2021
3,Liga Portugal,https://www.transfermarkt.co.uk/enzo-martinez/...,Enzo Martínez,Centre-Back,"Apr 29, 1998 (23)",Uruguay,"Sep 29, 2020","Dec 31, 2024",€500k,CD Tondela,https://www.transfermarkt.co.uk/cd-tondela/sta...,https://www.transfermarkt.co.uk/liga-portugal/...,2021
4,Liga Portugal,https://www.transfermarkt.co.uk/ricardo-alves/...,Ricardo Alves,Centre-Back,"May 9, 1991 (30)",Portugal,"Aug 30, 2018","Jun 30, 2025",€400k,CD Tondela,https://www.transfermarkt.co.uk/cd-tondela/sta...,https://www.transfermarkt.co.uk/liga-portugal/...,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8482,Süper Lig,https://www.transfermarkt.co.uk/ramazan-civele...,Ramazan Civelek,Right Winger,"Jan 22, 1996 (25)",Turkey,"Aug 5, 2019","Jun 30, 2025",€800k,Fatih Karagümrük,https://www.transfermarkt.co.uk/fatih-karagumr...,https://www.transfermarkt.co.uk/super-lig/star...,2021
8483,Süper Lig,https://www.transfermarkt.co.uk/jeremain-lens/...,Jeremain Lens,Right Winger,"Nov 24, 1987 (33)",Netherlands,"Feb 1, 2021","Jun 30, 2024",€600k,Fatih Karagümrük,https://www.transfermarkt.co.uk/fatih-karagumr...,https://www.transfermarkt.co.uk/super-lig/star...,2021
8484,Süper Lig,https://www.transfermarkt.co.uk/kemal-ademi/pr...,Kemal Ademi,Centre-Forward,"Jan 23, 1996 (25)",Switzerland,"Feb 1, 2021","Jun 30, 2023",€1.60m,Fatih Karagümrük,https://www.transfermarkt.co.uk/fatih-karagumr...,https://www.transfermarkt.co.uk/super-lig/star...,2021
8485,Süper Lig,https://www.transfermarkt.co.uk/artur-sobiech/...,Artur Sobiech,Centre-Forward,"Jun 12, 1990 (31)",Poland,"Jan 15, 2020","Jun 30, 2024",€1.30m,Fatih Karagümrük,https://www.transfermarkt.co.uk/fatih-karagumr...,https://www.transfermarkt.co.uk/super-lig/star...,2021


In [5]:
#Reformatting how club are written
players["Club"] =[unidecode(w) for w in players["Club"]]

players["Player"] =[unidecode(w) for w in players["Player"]]

In [6]:
players["Player"] = [re.sub(' +', ' ', w) for w in players["Player"]]

#### Transforming Values

In [7]:
#Getting the number of days left on contract
Contract_end = []

for w in players["Contract"] : 
    try:
        length = datetime.strptime(str(w), '%b %d, %Y') - datetime(2022,7,1,0,0,0,0)
        Contract_end.append(length.days)  
    except (ValueError, AttributeError): 
        Contract_end.append("NA")

players["Contract"] = Contract_end

In [8]:
#Getting the Age
Age = []

for row in players["Date of Birth"] :
    try : 
        Age.append(re.search(r'\((.*?)\)',row[row.rindex(',')+1:]).group(1))
    except (ValueError, AttributeError): 
        Age.append("NA")
        
players["Date of Birth"] = Age

players = players.rename(columns={'Date of Birth': 'Age'})

In [9]:
#Number of days since they joined the club
Longetivity = []
for w in players["Join"] : 
    try:
        length = datetime(2022,7,1,0,0,0,0) - datetime.strptime(str(w), '%b %d, %Y')
        Longetivity.append(length.days) 
    except (ValueError, AttributeError): 
        Longetivity.append("NA")

players["Join"] = Longetivity

In [10]:
#Converting to numeric
players[['Age', 'Join', 'Contract']] = players[['Age', 'Join', 'Contract']].apply(pd.to_numeric, errors='coerce', axis=1)

In [11]:
#Transforming the market value column
def value_to_float(x):
    x = str(x).replace('€', '')
    if 'k' in x:
        return float(x.replace('k', '')) / 1000
    if 'm' in x:
        return float(x.replace('m', ''))
    return 0

In [12]:
#Formatting
players['Market Value'] = players['Market Value'].apply(value_to_float)

In [13]:
names = ["Alisson", "Neymar", "Denilson", "Vitinho", "Martinelli", "Hulk", "Koke", "Raphinha", "Casemiro", "Marcelo", "Jorginho", "Ederson", "Willian", "Antony", "Marquinhos", "Thiago"]

for name in names : 
    row = players.loc[players["Player"] == name,].sort_values('Market Value', ascending=False).drop_duplicates('Player').iloc[:,:]
    players = players.loc[players["Player"] != name,]
    players = pd.concat([players, row], axis=0, ignore_index=True)

In [14]:
#In case of January transfer/ loan keep the original club
players = players.sort_values('Join', ascending=False).drop_duplicates('Player').sort_index()

#### Dropping Columns

In [15]:
#Contract Column dropped even though I could be kept however contract is new contract
players = players.drop([ "Season", "Club Link", "Contract"], axis=1)

#### Club Cleaning

In [16]:
players["Club"] = [l.replace('FC', '') for l in players["Club"]]

In [17]:
players["Club"] = [l.replace('Wolverhampton Wanderers', 'Wolves') for l in players["Club"]]

In [18]:
players["Club"] = [l.replace('Olympique', '') for l in players["Club"]]

In [19]:
players["Club"] = [l.replace('Hertha BSC', 'Hertha') for l in players["Club"]]

In [20]:
players.to_csv('Players_2021.csv', encoding='utf-8')